In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long endless immensity of the sea.")

seq_length = 10

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

dataX = []
dataY = []

for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1:i + seq_length + 1]
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)

In [4]:
g1 = tf.Graph()
with g1.as_default() as graph:
    
    #dic_size = len(char2idx)
    #rnn_hidden_size = len(char2idx)
    #num_classes = len(char2idx)
    #batch_size = 1
    #sequence_length = len(sample) - 1
    
    data_dim = len(char_set)
    rnn_hidden_size = len(char_set)
    num_classes = len(char_set)
    sequence_length = seq_length
    
    batch_size = len(dataX)
    
    X = tf.compat.v1.placeholder(tf.int32, [None, sequence_length])
    X_one_hot = tf.one_hot(X, num_classes)
    Y = tf.compat.v1.placeholder(tf.int32, [None, sequence_length])
    
    cell = tf.compat.v1.nn.rnn_cell.LSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
    cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell([cell] * 2, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    outputs, _states = tf.compat.v1.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)    
    
    ###SOFTMAX###
    X_for_softmax = tf.reshape(outputs, [-1, rnn_hidden_size])
    
    softmax_w = tf.compat.v1.get_variable("softmax_w", [rnn_hidden_size, num_classes])
    softmax_b = tf.compat.v1.get_variable("softmax_b", [num_classes])
    
    outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b    
    outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
    
    weights = tf.ones([batch_size, sequence_length])
    sequence_loss = tfa.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
    loss = tf.reduce_mean(sequence_loss)
    train = tf.compat.v1.train.AdamOptimizer(learning_rate=0.1).minimize(loss)    
    
    prediction = tf.argmax(outputs, axis=2)
    
    init_op = tf.compat.v1.global_variables_initializer()
    
sess = tf.compat.v1.Session(graph = g1)
sess.run(init_op)

In [7]:
for i in range(500):
    l, _ = sess.run([loss, train], feed_dict={X: dataX, Y: dataY})
    results = sess.run(prediction, feed_dict={X: dataX})
    
    if i % 100 == 0:    
        for j, result in enumerate(results):
            print(i, j, ''.join([char_set[t] for t in result]), l)
            
results = sess.run(prediction, feed_dict={X: dataX})
for j, result in enumerate(results):
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in result]), end='')
    else:
        print(char_set[result[-1]], end='')

0 0 p you want 0.22419518
0 1  you want  0.22419518
0 2 tou want t 0.22419518
0 3  u want to 0.22419518
0 4 n want to  0.22419518
0 5 pwant to b 0.22419518
0 6 tont to bu 0.22419518
0 7 ont to bui 0.22419518
0 8 nd to buil 0.22419518
0 9 d to build 0.22419518
0 10 hdo build  0.22419518
0 11 to luild a 0.22419518
0 12 h luild a  0.22419518
0 13 nbuild a s 0.22419518
0 14 tutld a sh 0.22419518
0 15 utld a shi 0.22419518
0 16 pld a ship 0.22419518
0 17 pd a ship, 0.22419518
0 18 e a ship,  0.22419518
0 19  anship, d 0.22419518
0 20 tnship, do 0.22419518
0 21 nship, don 0.22419518
0 22 thip, don' 0.22419518
0 23 sip, don't 0.22419518
0 24 ep, don't  0.22419518
0 25 p, don't d 0.22419518
0 26 , don't dr 0.22419518
0 27  don't dru 0.22419518
0 28 ton't drum 0.22419518
0 29  n't drum  0.22419518
0 30 n't drum u 0.22419518
0 31 dt drum up 0.22419518
0 32 t drum up  0.22419518
0 33 hdrum up p 0.22419518
0 34 toum up pe 0.22419518
0 35  um up peo 0.22419518
0 36  m up peop 0.22419518
0 37 p up p

200 0 f you want 0.2233584
200 1  you want  0.2233584
200 2 tou want t 0.2233584
200 3 ou want to 0.2233584
200 4 n want to  0.2233584
200 5  want to b 0.2233584
200 6 tont to bu 0.2233584
200 7 ont to bui 0.2233584
200 8 nd to buil 0.2233584
200 9 d to build 0.2233584
200 10  ro build  0.2233584
200 11 to build a 0.2233584
200 12   build a  0.2233584
200 13 nbuild a s 0.2233584
200 14 tutld a sh 0.2233584
200 15 uild a shi 0.2233584
200 16  ld a ship 0.2233584
200 17 fd a ship, 0.2233584
200 18 e a ship,  0.2233584
200 19  anship, d 0.2233584
200 20 tnship, do 0.2233584
200 21 nship, don 0.2233584
200 22 thip, don' 0.2233584
200 23 iip, don't 0.2233584
200 24 ep, don't  0.2233584
200 25 f, don't d 0.2233584
200 26   don't dr 0.2233584
200 27  don't dru 0.2233584
200 28 ton't drum 0.2233584
200 29  n't drum  0.2233584
200 30 n't drum u 0.2233584
200 31 dt drum up 0.2233584
200 32 t drum up  0.2233584
200 33  rrum up p 0.2233584
200 34 toum up pe 0.2233584
200 35  um up peo 0.2233584
20

400 0 t you want 0.22356978
400 1  you want  0.22356978
400 2 tou want t 0.22356978
400 3 ou want to 0.22356978
400 4 n want to  0.22356978
400 5  want to b 0.22356978
400 6 tont to bu 0.22356978
400 7 ont to bui 0.22356978
400 8 nd to buil 0.22356978
400 9 d to build 0.22356978
400 10  ro build  0.22356978
400 11 to cuild a 0.22356978
400 12   build a  0.22356978
400 13 ncuild a s 0.22356978
400 14 tutld a sh 0.22356978
400 15 uild a shi 0.22356978
400 16  ld a ship 0.22356978
400 17 td a ship, 0.22356978
400 18 e a ship,  0.22356978
400 19  anship, d 0.22356978
400 20 tnship, do 0.22356978
400 21 nship, don 0.22356978
400 22 thip, don' 0.22356978
400 23 iip, don't 0.22356978
400 24 ep, don't  0.22356978
400 25 t, don't d 0.22356978
400 26 e don't dr 0.22356978
400 27  bon't dru 0.22356978
400 28 ton't arum 0.22356978
400 29  n't arum  0.22356978
400 30 n't arum u 0.22356978
400 31 dt arum up 0.22356978
400 32 t arum up  0.22356978
400 33  rrum up p 0.22356978
400 34 toum up pe 0.2235